 # Notebook description

 In this notebook database views are created which will be helpful when doing reports in dash

 # Business requirements
 1. Top 10 lowest delays ratio
 2. Top 10 highest delays ratio 
 3. 2019 vs 2020 comparison to see if covid had any influence. 
     - month to month comparison
     - day to day comparison
 4. Daily delays ratio

 Libraries import and loading enviroment variables

In [1]:
from psycopg2 import connect
from dotenv import load_dotenv
from os import environ

In [2]:
dotenv_path = '../.env'
load_dotenv(dotenv_path)

True

In [3]:
username = environ.get('db_username')
password = environ.get('db_password')

host = environ.get('db_host')
database = environ.get('db_database')

Creating connection variables

In [4]:
con = connect(user=username,
                password=password,
                host=host,
                database=database)
cursor = con.cursor()

Loading 'reporting.sql' file

In [5]:
sql_code = ''
with open(r'..\sql\reporting.sql', 'r') as my_sql:
    code = my_sql.read().replace('\n', ' ')
    sql_code = f"{sql_code}{code}"
print(sql_code)

/* Defining `reporting` schema */ CREATE SCHEMA IF NOT EXISTS reporting ; /* Defining reporting.flight view, which: - doesn't include cancelled flights - include 'is_delayed' column */ CREATE OR REPLACE VIEW reporting.flight as SELECT *,        CASE            WHEN dep_delay_new > 0 THEN 1            ELSE 0        END AS is_delayed FROM flight where cancelled = 0  ; /* Defining reporting.top_reliability_roads view, which includes: - `origin_airport_id`, - `origin_airport_name`, - `dest_airport_id`, - `dest_airport_name`, - `year`, - `cnt` - number of flights on route, - `reliability` - delay ratio in route, - `nb` - ranking reliability,  columns */ CREATE OR REPLACE VIEW reporting.top_reliability_roads AS select f.origin_airport_id, 	al.name as origin_airport_name, 	f.dest_airport_id, 	al2.name as dest_airport_name, 	year, 	count(*) as cnt, 	round(is_delayed_mean, 2) as reliability, 	dense_rank() over(order by round(is_delayed_mean, 2)) as nb FROM flight f join airport_list al on f.ori

In [6]:
sql_code = sql_code.split(';')
sql_code

['/* Defining `reporting` schema */ CREATE SCHEMA IF NOT EXISTS reporting ',
 " /* Defining reporting.flight view, which: - doesn't include cancelled flights - include 'is_delayed' column */ CREATE OR REPLACE VIEW reporting.flight as SELECT *,        CASE            WHEN dep_delay_new > 0 THEN 1            ELSE 0        END AS is_delayed FROM flight where cancelled = 0  ",
 ' /* Defining reporting.top_reliability_roads view, which includes: - `origin_airport_id`, - `origin_airport_name`, - `dest_airport_id`, - `dest_airport_name`, - `year`, - `cnt` - number of flights on route, - `reliability` - delay ratio in route, - `nb` - ranking reliability,  columns */ CREATE OR REPLACE VIEW reporting.top_reliability_roads AS select f.origin_airport_id, \tal.name as origin_airport_name, \tf.dest_airport_id, \tal2.name as dest_airport_name, \tyear, \tcount(*) as cnt, \tround(is_delayed_mean, 2) as reliability, \tdense_rank() over(order by round(is_delayed_mean, 2)) as nb FROM flight f join airport

Executing querries from 'reporting.sql'

In [7]:
for query in sql_code:
    print(query)
    cursor.execute(query)

/* Defining `reporting` schema */ CREATE SCHEMA IF NOT EXISTS reporting 
 /* Defining reporting.flight view, which: - doesn't include cancelled flights - include 'is_delayed' column */ CREATE OR REPLACE VIEW reporting.flight as SELECT *,        CASE            WHEN dep_delay_new > 0 THEN 1            ELSE 0        END AS is_delayed FROM flight where cancelled = 0  
 /* Defining reporting.top_reliability_roads view, which includes: - `origin_airport_id`, - `origin_airport_name`, - `dest_airport_id`, - `dest_airport_name`, - `year`, - `cnt` - number of flights on route, - `reliability` - delay ratio in route, - `nb` - ranking reliability,  columns */ CREATE OR REPLACE VIEW reporting.top_reliability_roads AS select f.origin_airport_id, 	al.name as origin_airport_name, 	f.dest_airport_id, 	al2.name as dest_airport_name, 	year, 	count(*) as cnt, 	round(is_delayed_mean, 2) as reliability, 	dense_rank() over(order by round(is_delayed_mean, 2)) as nb FROM flight f join airport_list al on f.ori

 Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków

In [8]:
con.commit()

 ### Checking if the views were created

In [9]:
def check_if_table_exists(table_name):
    msg = f"Checking if {table_name} table exists"
    print(msg)

    query = f"select 1 from {table_name}"
    
    cursor.execute(query)
    print('OK!')

In [10]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [11]:
for view in views_to_test:
    check_if_table_exists(view)

Checking if reporting.flight table exists


OK!
Checking if reporting.top_reliability_roads table exists
OK!
Checking if reporting.year_to_year_comparision table exists
OK!
Checking if reporting.day_to_day_comparision table exists
OK!
Checking if reporting.day_by_day_reliability table exists
OK!


In [12]:
con.close()